In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mkman\\OneDrive\\Desktop\\Trail_and_Error\\End_to_end_Ml_Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\mkman\\OneDrive\\Desktop\\Trail_and_Error'

In [6]:
%cd End_to_end_Ml_Project

c:\Users\mkman\OneDrive\Desktop\Trail_and_Error\End_to_end_Ml_Project


In [7]:
%pwd

'c:\\Users\\mkman\\OneDrive\\Desktop\\Trail_and_Error\\End_to_end_Ml_Project'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from mlProject.constant import *
from mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config
 
        

In [13]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 

except Exception as e:
    raise e


[2025-08-31 19:44:36,250: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-08-31 19:44:36,252: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-08-31 19:44:36,257: INFO: common: yaml file: schema.yaml loaded succesfully]
[2025-08-31 19:44:36,262: INFO: common: created directory at: artifacts]
[2025-08-31 19:44:36,264: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-31 19:44:37,909: INFO: 3172890243: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4CF8:55387:1F7F51:506BFF:68B458CC
Accept-Ranges: bytes
Date: Sun, 31 Aug